<a href="https://colab.research.google.com/github/pedrorostagno/tesis/blob/main/Tesis_nuevo2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Usar A100, es el que mejor resultados me dio hasta ahora

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

files = os.listdir('/content/drive/Othercomputers/My Mac/Data/train/train/spoof/part_000')
print(len(files))  # Si esto ya crashea, ya encontraste el cuello de botella


1000


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import os
from PIL import Image
import numpy as np
import pickle
from datetime import datetime

dataset_path = '/content/drive/Othercomputers/My Mac/Data/train'

# 🗂️ Carpetas base en Drive
BASE_DIR = "/content/drive/MyDrive/Tesis UTDT"
CACHE_DIR = os.path.join(BASE_DIR, "cache")
EXPERIMENTS_DIR = os.path.join(BASE_DIR, "experimentos")

# ⏱️ Nombre único del experimento basado en fecha y hora
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
RUN_DIR = os.path.join(EXPERIMENTS_DIR, timestamp)

# 📁 Crear carpetas si no existen
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(RUN_DIR, exist_ok=True)

print(f"📁 Cachés en:     {CACHE_DIR}")
print(f"📁 Resultados en: {RUN_DIR}")


📁 Cachés en:     /content/drive/MyDrive/Tesis UTDT/cache
📁 Resultados en: /content/drive/MyDrive/Tesis UTDT/experimentos/20250701-210846


In [ ]:
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
import random

class CelebASpoofDatasetFromCSV(Dataset):
    def __init__(self, csv_path, transform=None, max_samples=None,
                 shuffle=True, seed=42, balanced=False):
        """
        Dataset desde CSV con opción de balanceo.

        CSV esperado con columnas: 'path', 'label'
        - label: 0 (spoof), 1 (live)
        """
        self.transform = transform
        df = pd.read_csv(csv_path)
        random.seed(seed)

        if balanced:
            spoof_df = df[df['label'] == 0]
            live_df  = df[df['label'] == 1]
            min_len = min(len(spoof_df), len(live_df))

            spoof_df = spoof_df.sample(n=min_len, random_state=seed)
            live_df  = live_df.sample(n=min_len, random_state=seed)
            df = pd.concat([spoof_df, live_df], ignore_index=True)

            print(f"⚖️ Dataset balanceado: {min_len} spoof + {min_len} live = {2 * min_len} total")

        if shuffle:
            df = df.sample(frac=1, random_state=seed).reset_index(drop=True)

        if max_samples:
            df = df.head(max_samples)

        self.image_paths = df['path'].tolist()
        self.labels = df['label'].tolist()

        print(f"📄 CSV cargado: {csv_path}")
        print(f"📊 Total imágenes cargadas: {len(self.image_paths)}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label


In [ ]:

# 📁 Definición de rutas a cada partición del dataset (entrenamiento, validación y test)
train_data_dir = '/content/drive/Othercomputers/My Mac/Data/train/train'
val_data_dir   = '/content/drive/Othercomputers/My Mac/Data/train/val'
test_data_dir  = '/content/drive/Othercomputers/My Mac/Data/train/test'

# 📐 Dimensiones estándar a las que se redimensionarán todas las imágenes
img_width, img_height = 224, 224

# ⚙️ Tamaño del batch para los DataLoaders
batch_size = 32

# 🌀 Transformaciones de preprocesamiento para cada conjunto de datos
data_transforms = {
    # 'train': transforms.Compose([
    #     transforms.Resize((img_width, img_height)),              # Redimensiona la imagen a 224x224
    #     transforms.RandomHorizontalFlip(),                       # Aplica flip horizontal aleatorio (augmentación)
    #     transforms.ToTensor(),                                   # Convierte PIL Image a tensor
    #     transforms.Normalize([0.485, 0.456, 0.406],              # Normalización con media y desvío estándar de ImageNet
    #                          [0.229, 0.224, 0.225])
    # ]),
    'train': transforms.Compose([
        transforms.Resize((img_width + 32, img_height + 32)),    # Aumenta resolución previa a crop
        transforms.RandomResizedCrop((img_width, img_height), scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.02),
        transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.2),
        transforms.RandomApply([transforms.RandomRotation(10)], p=0.3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}


In [ ]:
# Cantidad máxima de muestras por partición
train_max_samples = 100000
val_max_samples   = 20000
test_max_samples  = 5000

# 📦 Dataset de entrenamiento desde CSV
train_dataset = CelebASpoofDatasetFromCSV(
    csv_path='/content/drive/Othercomputers/My Mac/Data/train/train_image_list.csv',
    transform=data_transforms['train'],
    max_samples=train_max_samples,
    balanced=True
)

# 📦 Dataset de validación desde CSV
val_dataset = CelebASpoofDatasetFromCSV(
    csv_path='/content/drive/Othercomputers/My Mac/Data/train/val_image_list.csv',
    transform=data_transforms['val'],
    max_samples=val_max_samples,
    balanced=True
)

# 📦 Dataset de test desde CSV
# test_dataset = CelebASpoofDatasetFromCSV(
#     csv_path='/content/drive/Othercomputers/My Mac/Data/train/test_image_list.csv',
#     transform=data_transforms['test'],
#     max_samples=test_max_samples
# )

# 🔄 DataLoaders
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=10,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=10,
    pin_memory=True
)

# test_loader = DataLoader(
#     test_dataset,
#     batch_size=batch_size,
#     shuffle=False,
#     num_workers=8,
#     pin_memory=True
# )


⚖️ Dataset balanceado: 106825 spoof + 106825 live = 213650 total
📄 CSV cargado: /content/drive/Othercomputers/My Mac/Data/train/train_image_list.csv
📊 Total imágenes cargadas: 100000
⚖️ Dataset balanceado: 23863 spoof + 23863 live = 47726 total
📄 CSV cargado: /content/drive/Othercomputers/My Mac/Data/train/val_image_list.csv
📊 Total imágenes cargadas: 20000


In [ ]:
labels = np.array(train_dataset.labels)
unique, counts = np.unique(labels, return_counts=True)
print(f"Distribución de clases en entrenamiento: {dict(zip(unique, counts))}")

Distribución de clases en entrenamiento: {np.int64(0): np.int64(49785), np.int64(1): np.int64(50215)}


In [ ]:
# ─── 1) MODELO  ───────────────────────────────────────────────
from torchvision import models                     # ← ya lo usabas

model = models.mobilenet_v2(pretrained=True)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)       # 2 clases

# (opcional) descongela todo el backbone para fine-tuning completo
for p in model.parameters():
    p.requires_grad = True


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import os, json, torch, sklearn.metrics as metrics
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch import amp                      # 👈  nueva forma
from collections import OrderedDict

# ──────────────────────────────────────────────────────────────
#  1) FUNCIÓN DE PÉRDIDA – FOCAL LOSS
# ──────────────────────────────────────────────────────────────
class FocalLoss(nn.Module):
    def __init__(self, alpha=(0.25, 0.75), gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = torch.tensor(alpha)
        self.gamma = gamma
        self.reduction = reduction
        self.ce = nn.CrossEntropyLoss(reduction='none')

    def forward(self, logits, targets):
        # mueve alpha al device del tensor de logits
        if self.alpha.device != logits.device:
            self.alpha = self.alpha.to(logits.device)

        ce_loss = self.ce(logits, targets)
        pt      = torch.exp(-ce_loss)
        at      = self.alpha.gather(0, targets)
        loss    = at * (1 - pt) ** self.gamma * ce_loss
        return loss.mean() if self.reduction == 'mean' else loss

# ──────────────────────────────────────────────────────────────
# 2) BUCLE DE ENTRENAMIENTO (precisión mixta moderna)
# ──────────────────────────────────────────────────────────────
def train_epoch(model, loader, optimizer, criterion, device, scaler):
    model.train()
    loss_sum, correct, total = 0.0, 0, 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()

        with amp.autocast(device_type='cuda'):
            logits = model(x)
            loss   = criterion(logits, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        loss_sum += loss.item() * x.size(0)
        correct  += (logits.argmax(1) == y).sum().item()
        total    += x.size(0)

    return loss_sum / total, correct / total


def validate_epoch(model, loader, criterion, device):
    model.eval()
    loss_sum, correct, total = 0.0, 0, 0
    y_true, y_pred = [], []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            with amp.autocast(device_type='cuda'):
                logits = model(x)
                loss   = criterion(logits, y)

            loss_sum += loss.item() * x.size(0)
            correct  += (logits.argmax(1) == y).sum().item()
            total    += x.size(0)

            y_true.extend(y.cpu().numpy())
            y_pred.extend(logits.argmax(1).cpu().numpy())

    report = metrics.classification_report(
        y_true, y_pred, output_dict=True, zero_division=0
    )
    return loss_sum / total, correct / total, report

# ──────────────────────────────────────────────────────────────
# 3) FUNCIÓN PRINCIPAL DE ENTRENAMIENTO
# ──────────────────────────────────────────────────────────────
def train_model(
        model, train_loader, val_loader,
        criterion, optimizer, scheduler, device,
        num_epochs=20, patience=6, checkpoint_interval=1, log_dir=None):

    if log_dir is None:
        log_dir = os.path.join(RUN_DIR, "tensorboard")
    os.makedirs(log_dir, exist_ok=True)
    writer = SummaryWriter(log_dir)

    # guardar config
    json.dump(OrderedDict(
        num_epochs=num_epochs,
        patience=patience,
        batch_size=train_loader.batch_size,
        optimizer=type(optimizer).__name__,
        lr_start=optimizer.param_groups[0]["lr"]
    ), open(os.path.join(RUN_DIR, "config.json"), "w"), indent=2)

    scaler          = amp.GradScaler()
    best_val_loss   = float('inf')
    wait            = 0

    for epoch in range(1, num_epochs + 1):
        print(f"\nEpoch {epoch}/{num_epochs}\n" + "-" * 20)

        tr_loss, tr_acc = train_epoch(model, train_loader, optimizer,
                                      criterion, device, scaler)
        val_loss, val_acc, report = validate_epoch(
            model, val_loader, criterion, device)

        scheduler.step()   # CosineAnnealingLR no necesita métricas

        print(f"LR: {scheduler.get_last_lr()[0]:.6f}  "
              f"TrainLoss: {tr_loss:.4f}  ValLoss: {val_loss:.4f}")
        print(f"Val Acc: {val_acc:.4f}  F1_spoof: {report['0']['f1-score']:.3f}")

        # logs tensorboard
        writer.add_scalars("Loss", {"train": tr_loss, "val": val_loss}, epoch)
        writer.add_scalars("Acc",  {"train": tr_acc , "val": val_acc }, epoch)

        # early-stopping
        if val_loss < best_val_loss:
            best_val_loss, wait = val_loss, 0
            torch.save(model.state_dict(), os.path.join(RUN_DIR, "best_model.pth"))
            print("✅  Nuevo mejor modelo guardado")
        else:
            wait += 1
            print(f"⏳  Sin mejora ({wait}/{patience})")
            if wait >= patience:
                print("🛑  Early stopping")
                break

        # checkpoint periódico
        if epoch % checkpoint_interval == 0:
            torch.save({"epoch": epoch,
                        "model": model.state_dict(),
                        "optim": optimizer.state_dict()},
                       os.path.join(RUN_DIR, f"ckpt_{epoch:02d}.pth"))
    writer.close()
    return model


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model  = model.to(device)                          # tu MobileNetV2 ya modificado

criterion  = FocalLoss()                           # ← nueva loss
optimizer  = optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)
scheduler  = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)


In [39]:
# continua del bloque anterior, pero lo pongo aca para tenerlo separado
# 🧪 Ejecución del entrenamiento (se asume que todo está definido)
trained_model = train_model(
        model, train_loader, val_loader,
        criterion, optimizer, scheduler, device,
        num_epochs=10, patience=6
)


Epoch 1/10
--------------------
LR: 0.000497  TrainLoss: 0.0099  ValLoss: 0.0027
Val Acc: 0.9922  F1_spoof: 0.992
✅  Nuevo mejor modelo guardado

Epoch 2/10
--------------------
LR: 0.000488  TrainLoss: 0.0064  ValLoss: 0.0021
Val Acc: 0.9936  F1_spoof: 0.994
✅  Nuevo mejor modelo guardado

Epoch 3/10
--------------------
LR: 0.000473  TrainLoss: 0.0052  ValLoss: 0.0026
Val Acc: 0.9914  F1_spoof: 0.991
⏳  Sin mejora (1/6)

Epoch 4/10
--------------------
LR: 0.000452  TrainLoss: 0.0042  ValLoss: 0.0030
Val Acc: 0.9939  F1_spoof: 0.994
⏳  Sin mejora (2/6)

Epoch 5/10
--------------------
LR: 0.000427  TrainLoss: 0.0038  ValLoss: 0.0018
Val Acc: 0.9947  F1_spoof: 0.995
✅  Nuevo mejor modelo guardado

Epoch 6/10
--------------------
LR: 0.000397  TrainLoss: 0.0034  ValLoss: 0.0012
Val Acc: 0.9969  F1_spoof: 0.997
✅  Nuevo mejor modelo guardado

Epoch 7/10
--------------------
LR: 0.000363  TrainLoss: 0.0029  ValLoss: 0.0020
Val Acc: 0.9936  F1_spoof: 0.994
⏳  Sin mejora (1/6)

Epoch 8/10

In [40]:
import numpy as np
import sklearn.metrics as metrics
import seaborn as sns
import matplotlib.pyplot as plt
import torch

# 🧠 Cargar el mejor modelo desde el directorio de la corrida actual
best_model = models.mobilenet_v2(pretrained=False)
num_ftrs = best_model.classifier[1].in_features
best_model.classifier[1] = nn.Linear(num_ftrs, 2)
best_model.load_state_dict(torch.load(os.path.join(RUN_DIR, 'best_model.pth')))
best_model = best_model.to(device)
print("✅ Mejor modelo cargado desde RUN_DIR")



def test_model(model, test_loader, criterion, device, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    model.eval()
    all_preds = []
    all_labels = []
    test_loss = 0.0
    total_samples = 0

    # Usar autocast para precisión mixta en la inferencia
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            test_loss += loss.item() * inputs.size(0)
            total_samples += inputs.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = test_loss / total_samples
    print(f'Test Loss: {avg_loss:.4f}')

    # Convertir a arrays de numpy
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Calcular matriz de confusión y reporte en formato dict
    cm = metrics.confusion_matrix(all_labels, all_preds)
    report = metrics.classification_report(all_labels, all_preds, target_names=["spoof", "live"], zero_division=0)
    report_dict = metrics.classification_report(all_labels, all_preds, target_names=["spoof", "live"], output_dict=True, zero_division=0)

    # Guardar métricas como JSON
    with open(os.path.join(output_dir, 'classification_report.json'), 'w') as f:
        json.dump(report, f, indent=4)
    print("📄 Reporte de clasificación guardado en JSON.")

    print("Matriz de Confusión:")
    print(cm)
    print("Reporte de Clasificación:")
    print(report)

    # Guardar matriz de confusión como imagen PNG
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["spoof", "live"], yticklabels=["spoof", "live"])
    plt.xlabel("Predicción")
    plt.ylabel("Real")
    plt.title("Matriz de Confusión")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'confusion_matrix.png'))
    plt.close()
    print("🖼️ Matriz de confusión guardada como imagen.")

    return avg_loss, report_dict, cm




✅ Mejor modelo cargado desde RUN_DIR


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [41]:
# Continuacion del bloque anterior. Testeo del modelo en otro dataset
test_data_dir2 = '/content/drive/Othercomputers/My Mac/Data/test'


# 📦 Dataset de test desde CSV
test_dataset = CelebASpoofDatasetFromCSV(
    csv_path='/content/drive/Othercomputers/My Mac/Data/train/test_image_list.csv',
    transform=data_transforms['test'],
    max_samples=10000
)


# DataLoaders
test_loader2 = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)


# Ejemplo de uso:
# test_loss, test_report, test_cm = test_model(model, test_loader2, criterion, device)
test_loss, test_report, test_cm = test_model(best_model, test_loader2, criterion, device, output_dir=RUN_DIR)



📄 CSV cargado: /content/drive/Othercomputers/My Mac/Data/train/test_image_list.csv
📊 Total imágenes cargadas: 10000


/tmp/ipython-input-40-328312363.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Test Loss: 0.0022
📄 Reporte de clasificación guardado en JSON.
Matriz de Confusión:
[[7259   96]
 [  11 2634]]
Reporte de Clasificación:
              precision    recall  f1-score   support

       spoof       1.00      0.99      0.99      7355
        live       0.96      1.00      0.98      2645

    accuracy                           0.99     10000
   macro avg       0.98      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000

🖼️ Matriz de confusión guardada como imagen.


In [42]:
# Continuacion del bloque anterior. Testeo del modelo en otro dataset
test_data_dir2 = '/content/drive/Othercomputers/My Mac/Data/test'


# 📦 Dataset de test desde CSV
test_dataset2 = CelebASpoofDatasetFromCSV(
    csv_path='/content/drive/Othercomputers/My Mac/Data/train/test2_image_list.csv',
    transform=data_transforms['test'],
    max_samples=10000
)


# DataLoaders
test_loader2 = DataLoader(test_dataset2, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)


# Ejemplo de uso:
# test_loss, test_report, test_cm = test_model(model, test_loader2, criterion, device)
test_loss, test_report, test_cm = test_model(best_model, test_loader2, criterion, device, output_dir=RUN_DIR)



📄 CSV cargado: /content/drive/Othercomputers/My Mac/Data/train/test2_image_list.csv
📊 Total imágenes cargadas: 10000


/tmp/ipython-input-40-328312363.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Test Loss: 0.1780
📄 Reporte de clasificación guardado en JSON.
Matriz de Confusión:
[[4081 2991]
 [   3 2925]]
Reporte de Clasificación:
              precision    recall  f1-score   support

       spoof       1.00      0.58      0.73      7072
        live       0.49      1.00      0.66      2928

    accuracy                           0.70     10000
   macro avg       0.75      0.79      0.70     10000
weighted avg       0.85      0.70      0.71     10000

🖼️ Matriz de confusión guardada como imagen.


In [ ]:
import os
import pandas as pd

def log_experiment_summary(run_dir,
                           train_loss, train_acc,
                           val_loss, val_acc, val_report,
                           test_loss, test_report,
                           train_max_samples, val_max_samples, test_max_samples):
    summary_path = os.path.join(EXPERIMENTS_DIR, "all_runs.csv")
    run_name = os.path.basename(run_dir)

    # Utilidad segura para extraer métricas, aunque una clase no esté presente
    def get_metric(report, class_name, metric):
        return round(report.get(class_name, {}).get(metric, 0.0), 4)

    summary_data = {
        "run_name": run_name,

        # Métricas de entrenamiento
        "train_loss": round(train_loss, 4),
        "train_acc": round(train_acc.item(), 4),

        # Métricas de validación
        "val_loss": round(val_loss, 4),
        "val_acc": round(val_acc.item(), 4),
        "f1_macro_val": get_metric(val_report, "macro avg", "f1-score"),
        "precision_spoof_val": get_metric(val_report, "0", "precision"),
        "recall_spoof_val": get_metric(val_report, "0", "recall"),
        "f1_spoof_val": get_metric(val_report, "0", "f1-score"),
        "precision_live_val": get_metric(val_report, "1", "precision"),
        "recall_live_val": get_metric(val_report, "1", "recall"),
        "f1_live_val": get_metric(val_report, "1", "f1-score"),

        # Métricas de test final
        "test_loss": round(test_loss, 4),
        "f1_macro_test": get_metric(test_report, "macro avg", "f1-score"),
        "precision_spoof_test": get_metric(test_report, "spoof", "precision"),
        "recall_spoof_test": get_metric(test_report, "spoof", "recall"),
        "f1_spoof_test": get_metric(test_report, "spoof", "f1-score"),
        "precision_live_test": get_metric(test_report, "live", "precision"),
        "recall_live_test": get_metric(test_report, "live", "recall"),
        "f1_live_test": get_metric(test_report, "live", "f1-score"),

        # Info de dataset
        "train_max_samples": train_max_samples,
        "val_max_samples": val_max_samples,
        "test_max_samples": test_max_samples
    }

    df = pd.DataFrame([summary_data])
    if os.path.exists(summary_path):
        df.to_csv(summary_path, mode='a', header=False, index=False)
    else:
        df.to_csv(summary_path, index=False)

    print(f"📝 Resumen de la corrida guardado en {summary_path}")


In [ ]:
test_report

{'spoof': {'precision': 0.9924892703862661,
  'recall': 0.3474178403755869,
  'f1-score': 0.514675198219502,
  'support': 5325.0},
 'live': {'precision': 0.5728859390363815,
  'recall': 0.9970053475935828,
  'f1-score': 0.7276559206931543,
  'support': 4675.0},
 'accuracy': 0.6511,
 'macro avg': {'precision': 0.7826876047113238,
  'recall': 0.6722115939845849,
  'f1-score': 0.6211655594563281,
  'support': 10000.0},
 'weighted avg': {'precision': 0.796324712980195,
  'recall': 0.6511,
  'f1-score': 0.6142436859759345,
  'support': 10000.0}}

In [ ]:
log_experiment_summary(
    run_dir=RUN_DIR,
    train_loss=train_loss,
    train_acc=train_acc,
    val_loss=val_loss,
    val_acc=val_acc,
    val_report=val_report,
    test_loss=test_loss,
    test_report=test_report,
    train_max_samples=train_max_samples,
    val_max_samples=val_max_samples,
    test_max_samples=test_max_samples
)


📝 Resumen de la corrida guardado en /content/drive/MyDrive/Tesis UTDT/experimentos/all_runs.csv


In [ ]:
# Mostrar 10 predicciones incorrectas con su path
for i, (img, label) in enumerate(test_dataset2):
    pred = model(img.unsqueeze(0).to(device)).argmax(dim=1).item()
    if pred != label:
        print(f"ERROR → Pred: {pred} | Real: {label} | Path: {test_dataset2.image_paths[i]}")
        if i > 10: break


ERROR → Pred: 1 | Real: 0 | Path: /content/drive/Othercomputers/My Mac/Data/test/5028/spoof/538544.png
